In [1]:
!sudo apt -y install python3-magic
!pip install tfds-nightly
!pip install augly

In [2]:
import itertools
import os

import matplotlib.pylab as plt
import numpy as np
import augly.image as imaugs

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import scipy
from sklearn.metrics import roc_auc_score
import pickle
import gc
import PIL
from tensorflow.keras.applications import *
import imagehash
from PIL import Image
from tensorflow.keras import mixed_precision

print("TF version:", tf.__version__)
print("Hub version:", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

In [3]:
BATCH_SIZE = 32

In [4]:
class ReAct(tf.keras.layers.Layer):
  def __init__(self, λ):
    super(ReAct, self).__init__()
    self.λ = λ
  def build(self, input_shape):
    pass
  def call(self, inputs):
    return tf.math.minimum(inputs, self.λ)

In [17]:
def get_model(ds, model_base, preprocessing, p=0.9, show_info=False):
    pos = -1
    while True:
        layer_fv = model_base.get_layer(index=pos)
        if layer_fv.output.shape[-1] != model_base.get_layer(index=-1).output.shape[-1]:
            break
        pos -= 1

    model_fv = tf.keras.Model(inputs=model_base.input, outputs=layer_fv.output)
    model_fv = tf.keras.Sequential([model_fv, tf.keras.layers.Flatten()])
    y = model_fv.predict(ds).flatten()
    y_pos = min(int(p*len(y)), len(y)-1)
    λ = np.partition(y, pos, axis=0)[y_pos]
    if show_info:
        print(f'λ: {λ}')
    
    inp = tf.keras.layers.Input(shape=model_fv.output.shape[1:])
    x = inp
    for p in range(pos+1, 0):
        layer = model_base.get_layer(index=p)
        try:
            if layer.activation == tf.keras.activations.softmax:
                layer.activation = tf.keras.activations.linear
                if show_info:
                    print('softmax layer removed')
        except:
            pass
        x = layer(x)
    
    last_block = tf.keras.Model(inputs=inp, outputs=x)
    last_block.compile('adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
    
    @tf.function(jit_compile=True)
    def prep_image(x):
        return preprocessing(255.0*x)

    layer_prep_image = tf.keras.layers.Lambda(prep_image)
    
    # feature vector
    model_react = tf.keras.Sequential([
        tf.keras.Input((224, 224, 3)),
        layer_prep_image,
        model_fv,
        ReAct(λ),
        last_block
    ])
    
    model_base = tf.keras.Sequential([
        tf.keras.Input((224, 224, 3)),
        layer_prep_image,
        model_base,
    ])

    model_react.compile('adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'], steps_per_execution=64)
    model_base.compile('adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'], steps_per_execution=64)
    fe_base = tf.keras.Sequential([
        model_fv,        
    ])
    
    fe_react = tf.keras.Sequential([
        model_fv,  
        ReAct(λ),
    ])
    
    return model_base, model_react, fe_base, fe_react

In [6]:
def get_batch_gradient(model, loss_object, x, y_true=None):
  x = tf.convert_to_tensor(x)
  with tf.GradientTape() as tape:
    tape.watch(x)
    y_pred = model(x)
    if y_true is None:
      loss = loss_object(y_pred)
    else:
      loss = loss_object(y_true, y_pred)
  gradient = tape.gradient(loss, x)
  del loss, loss_object, x
  return gradient.numpy()

def get_gradient(model, loss_object, x, y_true=None, batch_size=BATCH_SIZE):
  grad = []
  pos = 0
  while pos < len(x):
    if y_true is not None:
      y_true_batch = y_true[pos:pos+batch_size]
    else:
      y_true_batch = None
    grad.append(get_batch_gradient(model, loss_object, x[pos:pos+batch_size], y_true=y_true_batch))
    pos += batch_size
  return np.concatenate(grad, axis=0)

def get_softmax_score(model, x, t=1):
  y_pred = model.predict(x)/t
  y_pred = scipy.special.softmax(y_pred, axis=0)
  y_pred = np.max(y_pred, axis=1)
  return y_pred

def odin_perturbation(model, x, t, epsilon):
  def loss(y_pred):
    y_pred = y_pred / t
    y_pred =  tf.nn.softmax(y_pred, axis=-1)
    return tf.math.log(tf.math.reduce_max(y_pred, axis=0))
  g = get_gradient(model, loss, x.copy())
  x -= epsilon * np.sign(-g)
  return x

def get_odin_score(model, x, t=1, epsilon=0.004):
  x = odin_perturbation(model, x, t=t, epsilon=epsilon)
  y_pred = model.predict(x)/t
  y_pred = scipy.special.softmax(y_pred)
  y_pred = np.max(y_pred, axis=1)
  return y_pred

def get_energy_score(model, x):
  y_pred = model.predict(x)
  score = np.log(1e-6+np.sum(np.exp(y_pred), axis=1))
  return score

In [7]:
def test_accuracy(model):
    ds = tfds.load('imagenet_v2', split=f'test[:{N_IMAGES}]', as_supervised=True, try_gcs=True)
    ds = ds.map(lambda x, y: (tf.image.resize(tf.image.convert_image_dtype(x, tf.float32), (224, 224)), y)) # [0, 1]
    ds = ds.batch(BATCH_SIZE)
    print(model.evaluate(ds))

In [8]:
def create_adversarial_pattern(model, input_image, loss_object, batch_size=BATCH_SIZE):
    gradient = get_gradient(model, loss_object, input_image, y_true=None, batch_size=batch_size)
    signed_grad = tf.sign(gradient)
    return signed_grad

# Run

In [11]:
N_IMAGES = 16

In [18]:
# any model can be use
model = ResNet101V2(input_shape=(224, 224, 3), classifier_activation='linear')
preprocessing = resnet_v2.preprocess_input

In [14]:
# id dataset
ds_id = tfds.load('imagenette', split=f'validation[:{N_IMAGES}]', as_supervised=True, try_gcs=True)
ds_id = ds_id.map(lambda x, y: (tf.image.resize(tf.image.convert_image_dtype(x, tf.float32), (224, 224)))) # [0, 1]
ds_id = ds_id.batch(BATCH_SIZE)
x_id = np.concatenate([a.numpy() for a in ds_id], axis=0)
x_id.shape

In [15]:
ds_ood =  tf.data.Dataset.from_tensor_slices(np.clip(np.random.normal(size=(N_IMAGES, 224, 224, 3)).astype(np.float32), 0, 1))
ds_ood = ds_ood.batch(BATCH_SIZE)
x_ood = np.concatenate([a.numpy() for a in ds_ood], axis=0)
x_ood.shape

In [19]:
# generate the models
model_base, model_react, fe_base, fe_react = get_model(ds_id, model, preprocessing, p=0.9)

In [ ]:
# example id

print(f'without react, softmax: {get_softmax_score(model_base, x_id, t=1)}')
print(f'with react, softmax: {get_softmax_score(model_react, x_id, t=1)}')
print(f'ODIN: {get_odin_score(model_base, x_id, t=1000, epsilon=0.0014)}')
print(f'Energy: {get_odin_score(model_base, x_id, t=1000, epsilon=0.0014)}')

In [ ]:
# example ood

print(f'without react, softmax: {get_softmax_score(model_base, x_ood, t=1)}')
print(f'with react, softmax: {get_softmax_score(model_react, x_ood, t=1)}')
print(f'ODIN: {get_odin_score(model_base, x_ood, t=1000, epsilon=0.0014)}')
print(f'Energy: {get_odin_score(model_base, x_ood, t=1000, epsilon=0.0014)}')